
# Proyecto WWII — End-to-End (OFFLINE, dataset incluido)

Este proyecto usa un **dataset consolidado incluido** (`data/ww2_country_panel.csv`) con cifras de bajas, población, PIB per cápita (Maddison, 1990 intl $ aprox.), y producción industrial (aviones, tanques) 1939–1945 para 10 países principales.
> Nota: Las estimaciones varían según la fuente. Este snapshot busca ser **didáctico** y razonable para análisis educativo; ver README para contexto y referencias.

## 1) Setup


In [ ]:

import warnings; warnings.filterwarnings('ignore')
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

DATA_PATH = "data/ww2_country_panel.csv"
df = pd.read_csv(DATA_PATH)
df.head()


## 2) Calidad de datos y derivadas

In [ ]:

df.info()
print("\nNulos por columna:\n", df.isna().sum())
df.describe()


## 3) EDA

In [ ]:

# Muertes por 1000 habitantes (barra)
plt.figure(); 
order = df.sort_values('deaths_per_1000', ascending=False)
plt.bar(order['entity'], order['deaths_per_1000'])
plt.xticks(rotation=45, ha='right'); plt.ylabel('Muertes por 1000'); plt.title('Tasa de bajas WWII (estimación)'); plt.tight_layout(); plt.show()

# Producción de aviones vs muertes per cápita
plt.figure(); 
plt.scatter(df['aircraft_per_million_pop'], df['deaths_per_1000'])
for i, r in df.iterrows():
    plt.annotate(r['entity'], (r['aircraft_per_million_pop'], r['deaths_per_1000']), fontsize=8)
plt.xlabel('Aviones por millón de hab. (39–45)')
plt.ylabel('Muertes por 1000')
plt.title('Relación producción de aviones vs. bajas per cápita')
plt.tight_layout(); plt.show()

# Box de GDP per cápita antes y después
plt.figure(); 
plt.boxplot([df['gdp_pc_1939_int1990'], df['gdp_pc_1945_int1990']], labels=['1939','1945'])
plt.ylabel('GDP per cápita (1990 intl $ aprox.)'); plt.title('PIB per cápita: 1939 vs 1945')
plt.show()


## 4) Modelos didácticos

### 4.1 Regresión: explicar `deaths_per_1000`

In [ ]:

X = df[['gdp_pc_1939_int1990','aircraft_per_million_pop','tanks_per_million_pop','pop_1939_mill']].copy()
y = df['deaths_per_1000'].copy()

loo = LeaveOneOut()
preds, ys = [], []
for tr, te in loo.split(X):
    pipe = Pipeline([('sc', StandardScaler()), ('lr', LinearRegression())])
    pipe.fit(X.iloc[tr], y.iloc[tr])
    p = pipe.predict(X.iloc[te])[0]
    preds.append(p); ys.append(y.iloc[te].values[0])

mae = mean_absolute_error(ys, preds); r2 = r2_score(ys, preds)
print({'MAE': round(mae,3), 'R2_LOOCV': round(r2,3)})


### 4.2 Clasificación: alta vs. baja mortalidad per cápita

In [ ]:

thr = df['deaths_per_1000'].median()
df['high_mortality'] = (df['deaths_per_1000'] >= thr).astype(int)

Xc = df[['gdp_pc_1939_int1990','aircraft_per_million_pop','tanks_per_million_pop','pop_1939_mill']].copy()
yc = df['high_mortality'].copy()

loo = LeaveOneOut()
preds, ys = [], []
for tr, te in loo.split(Xc):
    pipe = Pipeline([('sc', StandardScaler()), ('clf', LogisticRegression(max_iter=500))])
    pipe.fit(Xc.iloc[tr], yc.iloc[tr])
    p = pipe.predict(Xc.iloc[te])[0]
    preds.append(p); ys.append(yc.iloc[te].values[0])

acc = accuracy_score(ys, preds); prec = precision_score(ys, preds); rec = recall_score(ys, preds); f1 = f1_score(ys, preds)
print({'acc': round(acc,3), 'prec': round(prec,3), 'rec': round(rec,3), 'f1': round(f1,3)})
print("Matriz de confusión:\n", confusion_matrix(ys, preds))


## 5) Exportables

In [ ]:

import os
os.makedirs("data/processed", exist_ok=True)
df.to_csv("data/processed/ww2_country_panel_enriquecido.csv", index=False)
print("Guardado data/processed/ww2_country_panel_enriquecido.csv")


## 6) Notas de gobernanza / límites


- Las cifras de bajas y producción WWII **varían por fuente**; este snapshot busca valores plausibles y ampliamente citados para uso educativo.
- Recomendado: validar contra fuentes académicas/museos y documentar la **fecha y fuente** para informes formales.
- Este proyecto es **offline** para evitar fallas de red/403/404.
